# Workshop SL02: Regression

## Agenda
- Common regression models

## Previously on the last workshops
From the last workshops we have covered: 
- Prepping data 
- Classification models

For this workshop we are gonna work with a new dataset (download it here) and try out different regression models.

## Exercise
- read data into dataframe
- prep the data for training 
- try out different regression model

### Prep work

In [1]:
# read data

### Model Training
List of regression models:
- Generalized Linear Models 
- Kernel Ridge Regression
- Support Vector Machines 
- Stochastic Gradient Descent 
- Nearest Neighbors
- Gaussian Processes
- Naive Bayes
- Trees
- Ensemble Methods 

Apart from single output regression, there is a more difficult regression problem when we need to return multiple outputs. Then we can use `MultiOutputRegressor` with any regressor. More details [here](https://scikit-learn.org/stable/modules/multiclass.html#multioutput-regression).


#### !) GLM